<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 4</font>
## <font color='blue'>Ajuste Fino de LLM Open Source Para Chatbot de Atendimento ao Cliente</font>

## Instalando e Carregando Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00


In [2]:
# Instalando os pacotes
!pip install -q bitsandbytes datasets accelerate loralib evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.4 MB/s eta 0:00:00


In [3]:
# Instalando os pacotes
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
# Imports
import os
import json
import torch
import evaluate
import torch.nn as nn
import transformers
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model
from datasets import Dataset, Features, ClassLabel, Value, Sequence
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [6]:
# Verifica a GPU
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo GPU:', torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory / 1e9)

Número de GPUs: 1
Modelo GPU: NVIDIA A100-SXM4-40GB
Total Memória [GB] da GPU: 42.474471424


## Definindo os Parâmetros de Quantização

In [7]:
# Define os parâmetros
quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                         bnb_4bit_compute_dtype = torch.float16,
                                         bnb_4bit_quant_type = "nf4",
                                         bnb_4bit_use_double_quant = True,
                                         llm_int8_enable_fp32_cpu_offload = True)

Os parâmetros acima são configurações específicas para a quantização de modelos de aprendizado de máquina com o PyTorch. Aqui está uma explicação para cada um deles:

**load_in_4bit (True):** Este parâmetro indica que o modelo deve ser carregado em um formato de 4 bits. Isso é  definido para reduzir o uso de memória, permitindo que modelos grandes sejam carregados em hardware com memória limitada. Quando True, o modelo é carregado com uma precisão reduzida.

**bnb_4bit_compute_dtype (torch.float16):** Define o tipo de dados a ser usado para cálculos internos quando o modelo está em formato de 4 bits. O torch.float16 indica que os cálculos serão realizados usando o tipo de dados de ponto flutuante de 16 bits, que é uma boa combinação de precisão e eficiência para muitos modelos de aprendizado de máquina.

**bnb_4bit_quant_type ("nf4"):** Especifica o tipo de quantização a ser aplicada. O "nf4" se refere a um tipo específico de quantização de 4 bits. Referência aqui: https://huggingface.co/blog/4bit-transformers-bitsandbytes

**bnb_4bit_use_double_quant (True):** Indica se deve ser usada uma "quantização dupla" durante o processo de quantização. A quantização dupla pode ajudar a manter mais precisão nos dados quantizados, mas pode ter um custo computacional mais alto.

**llm_int8_enable_fp32_cpu_offload (True):** Este parâmetro sugere que para operações de baixa latência (em modelos grandes como LLMs - Large Language Models), quando os cálculos estão sendo feitos em int8 (8 bits), há uma opção para descarregar (offload) algumas operações para o CPU em formato fp32 (ponto flutuante de 32 bits). Isso pode ser útil para equilibrar carga entre CPU e GPU e para lidar com operações que exigem maior precisão. Referência aqui: https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Esses parâmetros são avançados e específicos para otimizações de desempenho e uso de memória em modelos de aprendizado de máquina, especialmente modelos grandes que podem ter restrições de hardware

## Carregando Modelo e Tokenizador

https://huggingface.co/tiiuae/falcon-7b

In [8]:
# Modelo
modelo = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b",
                                             quantization_config = quantization_config,
                                             device_map = 'auto')

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [9]:
# Tokenizador
tokenizador = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

## Congelando os Pesos Originais

In [10]:
# Loop
for param in modelo.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

**param.requires_grad = False**: Isso desativa o cálculo de gradientes para cada parâmetro, o que significa que esses parâmetros não serão atualizados durante o treinamento. Isso é comumente feito quando se quer congelar certas partes de um modelo pré-treinado para transferência de aprendizado, onde apenas as camadas superiores do modelo serão treinadas.

**if param.ndim == 1: param.data = param.data.to(torch.float32)**: Esta linha converte os parâmetros unidimensionais (ou seja, vetores) para o tipo de dado float32. Isso pode ser necessário se for desejado garantir que todos os parâmetros do modelo estejam no mesmo tipo de dado, especialmente se forem realizadas operações que requerem consistência de tipos.

## Ativando o Checkpoint de Gradientes do Modelo

In [11]:
# Ativa o recurso de checkpoint de gradientes no modelo
modelo.gradient_checkpointing_enable()

Quando o checkpoint de gradientes está habilitado, o modelo não armazena todos os valores intermediários (atividades das camadas) durante a passagem para frente (forward pass). Em vez disso, ele armazena apenas alguns pontos de verificação. Durante a passagem para trás (backward pass), os valores intermediários que não foram armazenados são recomputados a partir dos pontos de verificação. Isso reduz a quantidade de memória necessária para armazenar os valores intermediários, mas aumenta o tempo de computação, pois alguns valores precisam ser recomputados.

Essa técnica é útil quando se treina modelos muito grandes que de outra forma não caberiam na memória da GPU.

In [12]:
# Habilita a técnica chamada "checkpointing de gradiente"
modelo.enable_input_require_grads()

Essa técnica é útil para reduzir o consumo de memória durante o treinamento de modelos grandes. O que ela faz é salvar certos estados intermediários (checkpoints) durante a passagem para frente (forward pass) e, em seguida, durante a passagem para trás (backward pass), esses estados são usados para recomputar os gradientes, em vez de armazenar todos os estados intermediários na memória. Isso pode diminuir a quantidade de memória necessária, mas pode aumentar o tempo de computação.

## Ajustando a Conversão Para Tensor

In [13]:
# Conversão de tensor
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

A classe CastOutputToFloat é uma subclasse da classe nn.Sequential do PyTorch, que é usada para criar uma sequência de módulos (como camadas de uma rede neural). A principal funcionalidade dessa classe é converter o tipo de dados da saída de uma sequência de módulos para torch.float32 (ou seja, um tensor de ponto flutuante de 32 bits).

In [14]:
modelo.lm_head = CastOutputToFloat(modelo.lm_head)

A linha acima está substituindo a cabeça de modelo de linguagem (lm_head) pelo módulo CastOutputToFloat que encapsula a cabeça de modelo de linguagem original. Isso significa que toda vez que a lm_head do modelo for utilizada durante a passagem para frente, sua saída será automaticamente convertida para torch.float32. Isso pode ser útil para garantir a compatibilidade de tipo em situações onde a saída da cabeça de modelo de linguagem precisa ser do tipo float32.

## Definindo os Parâmetros do Ajuste Fino

In [15]:
# LoRa Config
config = LoraConfig(r = 16,
                    lora_alpha = 32,
                    lora_dropout = 0.05,
                    bias = "none",
                    task_type = "CAUSAL_LM")

Os parâmetros dentro de LoraConfig ajustam diferentes aspectos da configuração de um modelo LoRA, que é uma técnica de ajuste fino para modelos de aprendizado profundo. Aqui está a explicação de cada um:

**r**: Este parâmetro especifica a taxa de redução, que afeta diretamente o tamanho dos parâmetros adaptativos adicionados ao modelo. Uma taxa de redução menor significa menos parâmetros a serem aprendidos, o que pode tornar o ajuste fino mais eficiente e rápido. Por outro lado, uma taxa de redução maior permite uma adaptação mais flexível ao novo conjunto de dados, mas com o custo de maior complexidade computacional.

**lora_alpha**: Define o parâmetro de escala para a adaptação LoRA. lora_alpha ajusta a magnitude da atualização aplicada aos pesos do modelo original. Um valor mais alto significa que as atualizações adaptativas terão mais impacto, permitindo ajustes mais significativos no comportamento do modelo. Essa escala pode ser crucial para garantir que as mudanças introduzidas sejam relevantes para a tarefa específica sem desviar demasiadamente do conhecimento pré-aprendido.

**lora_dropout**: A taxa de dropout aplicada aos parâmetros adaptativos LoRA. Dropout é uma técnica de regularização usada para prevenir o sobreajuste durante o treinamento. Ao definir lora_dropout, você especifica a probabilidade com que as conexões entre os neurônios adaptativos serão temporariamente desativadas. Isso ajuda o modelo a generalizar melhor para dados não vistos, promovendo a robustez do ajuste fino.

**bias**: Define se os termos de bias serão incluídos nos ajustes LoRA. Os termos de bias adicionam um valor constante às saídas de uma camada, o que pode ajudar na otimização e na capacidade do modelo de se ajustar aos dados. Configurar bias como "none" significa que esses ajustes não serão aplicados, o que pode ser preferível em cenários onde a inclusão de termos de bias não oferece benefícios claros ou se deseja manter a estrutura do modelo o mais simples possível.

**task_type**: Especifica o tipo de tarefa para a qual o modelo está sendo ajustado. "CAUSAL_LM" refere-se a Modelagem de Linguagem Causal, um cenário onde o modelo gera texto baseado no contexto anterior de forma sequencial. Isso é essencial para garantir que as adaptações LoRA sejam direcionadas para melhorar o desempenho nesse tipo específico de tarefa, otimizando o modelo para gerar respostas coerentes e contextuais.

In [16]:
# Cria o modelo considerando os parâmetros LoRa
modelo = get_peft_model(modelo, config)

In [17]:
# Definindo a função para imprimir os parâmetros treináveis de um modelo
def dsa_print_trainable_parameters(model):

    # Inicializa a contagem de parâmetros treináveis
    trainable_params = 0

    # Inicializa a contagem de todos os parâmetros
    all_param = 0

    # Itera sobre todos os parâmetros nomeados do modelo
    for _, param in model.named_parameters():

        # Soma o número total de elementos de todos os parâmetros
        all_param += param.numel()

        # Verifica se o parâmetro é treinável
        if param.requires_grad:

            # Soma o número de elementos aos parâmetros treináveis
            trainable_params += param.numel()

    # Imprime o número de parâmetros treináveis, o total de parâmetros e a porcentagem de parâmetros treináveis
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [18]:
# Executa a função
dsa_print_trainable_parameters(modelo)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


## Processamento dos Dados

In [19]:
# Define o arquivo
arquivo = open("dataset.json")

In [20]:
# Carrega o arquivo
dados_dsa = json.load(arquivo)

In [21]:
# Visualiza os dados
dados_dsa

{'perguntas': [{'pergunta': 'Como posso criar uma conta?',
   'resposta': 'Para criar uma conta, clique no botão ‘Cadastre-se’ no canto superior direito do nosso site e siga as instruções para concluir o processo de registro.'},
  {'pergunta': 'Que tipos de pagamentos você aceita?',
   'resposta': 'Aceitamos os principais cartões de crédito, cartões de débito e PayPal como métodos de pagamento para pedidos online.'},
  {'pergunta': 'Como posso rastrear meu pedido?',
   'resposta': 'Você pode acompanhar seu pedido fazendo login em sua conta e navegando até a seção ‘Histórico de pedidos’. Lá você encontrará as informações de rastreamento da sua remessa.'},
  {'pergunta': 'Qual é a sua política de devolução?',
   'resposta': 'Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.'},
  {'pergunta'

In [22]:
# Listas para perguntas e respostas
perguntas = []
respostas = []

In [23]:
# Loop pelos dados para extrair perguntas e respostas
for i in dados_dsa["perguntas"]:
    perguntas += [i["pergunta"]]
    respostas += [i["resposta"]]

In [24]:
# Conteúdo do dataset original
dados_dsa["perguntas"][0]

{'pergunta': 'Como posso criar uma conta?',
 'resposta': 'Para criar uma conta, clique no botão ‘Cadastre-se’ no canto superior direito do nosso site e siga as instruções para concluir o processo de registro.'}

In [25]:
# Uma pergunta
perguntas[0]

'Como posso criar uma conta?'

In [26]:
# Resposta associada à pergunta anterior
respostas[0]

'Para criar uma conta, clique no botão ‘Cadastre-se’ no canto superior direito do nosso site e siga as instruções para concluir o processo de registro.'

In [27]:
# Agora colocamos os dados no formato adequado para treinar o modelo
dataset = Dataset.from_dict({
    "id": list(range(len(perguntas))),
    "perguntas": perguntas,
    "respostas": respostas
    },
    features = Features({
        "id": Value(dtype = 'string'),
        "perguntas": Value(dtype = "string"),
        "respostas": Value(dtype = "string")
    }
))

In [28]:
# Divide os dados em treino e teste
dataset = dataset.train_test_split(test_size = 0.15)

In [29]:
# Função para o merge de colunas concatenando cada pergunta com a resposta correspondente
def dsa_merge_columns(registro):
    registro["saida"] = registro["perguntas"] + " ->: " + registro["respostas"]
    return registro

In [30]:
# Aplica a função
dsa_dataset = dataset.map(dsa_merge_columns)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [31]:
# Este será o formato dos dados para treinar o modelo
dsa_dataset["train"]["saida"][0]

'Qual é a sua política de devolução? ->: Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.'

In [32]:
# Observe que temos um id, temos a entrada (perguntas e respostas) e temos a saída (combinação de pergunta e resposta)
dsa_dataset["train"][0]

{'id': '3',
 'perguntas': 'Qual é a sua política de devolução?',
 'respostas': 'Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.',
 'saida': 'Qual é a sua política de devolução? ->: Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.'}

In [33]:
# Tokenizamos os dados
dsa_dataset = dsa_dataset.map(lambda samples: tokenizador(samples['saida']), batched = True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [34]:
# Dados tokenizados
dsa_dataset["train"][0]

{'id': '3',
 'perguntas': 'Qual é a sua política de devolução?',
 'respostas': 'Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.',
 'saida': 'Qual é a sua política de devolução? ->: Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.',
 'input_ids': [22892,
  1685,
  241,
  9828,
  34919,
  336,
  1612,
  32228,
  5372,
  42,
  204,
  1579,
  37,
  409,
  31764,
  34919,
  336,
  1612,
  32228,
  5372,
  30326,
  877,
  17200,
  1612,
  315,
  3095,
  64650,
  658,
  7179,
  5173,
  336,
  204,
  34,
  31687,
  55697,
  123,
  94,
  241,
  42641,
  293,
  1477,
  4975,
  292,
  357,
  4016,
 

## Definindo os Argumentos de Treino

In [35]:
# Se não tiver pad, ajustamos com o pad do tokenizador
if tokenizador.pad_token == None:
    tokenizador.pad_token = tokenizador.eos_token

In [36]:
# Argumentos de treino
dsa_trainer = transformers.Trainer(model = modelo,
                                   train_dataset = dsa_dataset["train"],
                                   eval_dataset = dsa_dataset["test"],
                                   args = transformers.TrainingArguments(eval_strategy = "epoch",
                                                                         per_device_train_batch_size = 2,
                                                                         gradient_accumulation_steps = 2,
                                                                         num_train_epochs = 10,
                                                                         learning_rate = 2e-4,
                                                                         fp16 = True,
                                                                         logging_steps = 1,
                                                                         output_dir = 'outputs',
                                                                         report_to = "none"),
                                   data_collator = transformers.DataCollatorForLanguageModeling(tokenizador, mlm = False))

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Aqui está a explicação dos componentes principais na configuração de treinamento usando a classe Trainer do transformers:

**model**: Este argumento especifica o modelo que será treinado. Aqui, modelo é uma instância de um modelo pré-carregado ou customizado que você deseja treinar ou ajustar fino para uma tarefa específica.

**train_dataset**: Define o conjunto de dados de treinamento. dsa_dataset["train"] indica que estamos utilizando a parte de treinamento de um conjunto de dados chamado dsa_dataset.

**eval_dataset**: Define o conjunto de dados de avaliação (teste). dsa_dataset["test"] refere-se à parte de teste do mesmo conjunto de dados, utilizado para avaliar o desempenho do modelo após cada época de treinamento, dependendo da estratégia de avaliação definida.

**args**: Uma instância de transformers.TrainingArguments que contém vários argumentos de configuração para o processo de treinamento:

**evaluation_strategy = "epoch"**: A avaliação do modelo acontece ao final de cada época.

**per_device_train_batch_size = 2**: Define o tamanho do lote (batch size) para o treinamento em cada dispositivo (por exemplo, GPU).

**gradient_accumulation_steps = 2**: O número de passos de acumulação de gradiente antes de realizar uma etapa de otimização. Isso efetivamente aumenta o tamanho do lote ao custo de memória reduzido, já que os gradientes são acumulados em várias etapas antes da atualização dos pesos.

**num_train_epochs = 5**: O número de épocas de treinamento.

**learning_rate = 2e-4**: A taxa de aprendizagem inicial.

**fp16 = True**: Habilita o treinamento usando precisão mista (floating point 16), o que pode acelerar o treinamento e reduzir o uso de memória em GPUs compatíveis.

**logging_steps = 1**: Frequência (em número de passos de treinamento) para logar as métricas de treinamento.

**output_dir = 'outputs'**: O diretório onde os artefatos de treinamento (como modelos salvos) serão armazenados.

**data_collator**: Especifica como os lotes de dados são formados ou colados juntos. transformers.DataCollatorForLanguageModeling é usado para a modelagem de linguagem, onde tokenizador é o tokenizador a ser usado para preparar os lotes de treinamento, e mlm = False indica que a modelagem de linguagem não é mascarada (ou seja, é uma modelagem de linguagem causal).

## Treinamento do Modelo

In [37]:
# Não usa o cache
modelo.config.use_cache = False

In [38]:
# Treinamento e ajuste fino do modelo
dsa_trainer.train()

Epoch,Training Loss,Validation Loss
1,1.661900,2.034001
2,1.792900,2.011462
3,1.731500,1.982294
4,1.584800,1.951303
5,1.449100,1.919328
6,1.453100,1.888561
7,1.326800,1.861743
8,1.168100,1.840833
9,1.622900,1.827485
10,1.216600,1.822236


TrainOutput(global_step=20, training_loss=1.5069120347499847, metrics={'train_runtime': 14.4295, 'train_samples_per_second': 3.465, 'train_steps_per_second': 1.386, 'total_flos': 139807376540160.0, 'train_loss': 1.5069120347499847, 'epoch': 10.0})

## Avaliação do Modelo

In [39]:
# Função para extrair as previsões
def dsa_predict(question):

    # Garante que não estamos em modo treino
    modelo.eval()

    # Identifica cpu ou cuda (GPU)
    device = next(modelo.parameters()).device

    # tokeniza e já envia para o mesmo dispositivo do modelo
    batch = tokenizador(f"{question} ->: ",
                        return_tensors='pt',
                        padding=True,
                        truncation=True)

    batch = {k: v.to(device) for k, v in batch.items()}

    # Sem gradiente e com autocast em GPU para fp16
    with torch.no_grad(), torch.cuda.amp.autocast():
        output_tokens = modelo.generate(
            **batch,
            max_new_tokens=50,
            pad_token_id=tokenizador.eos_token_id
        )

    # Decodifica de volta para string
    return tokenizador.decode(output_tokens[0], skip_special_tokens = True)

In [40]:
# Lista para as previsões
previsoes = []

In [41]:
# Loop para extrair as previsões
for i in dsa_dataset["test"]["perguntas"]:
    previsoes.append(dsa_predict(i))

## Interpretando a Métrica

In [42]:
# Carrega o módulo da métrica
bleu = evaluate.load('bleu')

A métrica BLEU (Bilingual Evaluation Understudy) é uma das métricas mais conhecidas e utilizadas em Processamento de Linguagem Natural (PLN) para avaliar a qualidade de textos gerados por máquinas. Desenvolvida por Papineni et al. em 2002, a métrica BLEU compara um texto gerado automaticamente com um ou mais textos de referência (geralmente humanos) para medir a qualidade da produção da máquina.

A essência da métrica BLEU é medir quão similares são os n-gramas (sequências contínuas de n itens de um dado texto) do texto gerado automaticamente em comparação aos n-gramas dos textos de referência. O cálculo da pontuação BLEU envolve os seguintes passos principais:

**Cálculo de Precisão de N-gramas**: Para cada n-grama no texto gerado, verifica-se quantos desses n-gramas também aparecem nos textos de referência. A precisão é então calculada como o número de n-gramas coincidentes dividido pelo número total de n-gramas no texto gerado.

**Punição por Sentenças Curtas (Brevity Penalty, BP)**: Para evitar que a métrica favoreça respostas indevidamente curtas (que podem ter alta precisão de n-gramas mas são inúteis), o BLEU inclui uma punição para textos gerados que são mais curtos que os textos de referência. Essa punição diminui a pontuação BLEU de textos muito curtos, equilibrando a precisão e a completude da resposta.

**Combinação de Precisões com Pesos**: A pontuação BLEU final geralmente combina as precisões de n-gramas de diferentes tamanhos (por exemplo, de 1 a 4), aplicando pesos a cada uma dessas precisões. Isso é feito para considerar a fluidez e a estrutura gramatical do texto, além da presença de palavras-chave.

A pontuação final do BLEU é então calculada usando uma média geométrica dessas precisões ponderadas, ajustada pela punição por sentenças curtas. A pontuação varia de 0 a 1, onde 1 indica uma correspondência perfeita com o texto de referência, embora frequentemente seja expressa em porcentagem (0 a 100).

Embora o BLEU seja amplamente utilizado devido à sua simplicidade e capacidade de avaliação rápida, ele tem limitações, como a incapacidade de avaliar a adequação semântica ou a fluidez gramatical de maneira mais profunda. Por isso, outras métricas complementares, como ROUGE, METEOR e outras, também são usadas para avaliar a qualidade da geração de texto em PLN.

In [43]:
# Extrai os dados reais
dados_reais = dsa_dataset["test"]["saida"]

In [44]:
# Calcula a métrica comparando valores reais e previsões
resultado = bleu.compute(predictions = previsoes, references = dados_reais)

In [45]:
resultado

{'bleu': 0.18274596140497432,
 'precisions': [0.2826086956521739,
  0.17777777777777778,
  0.1590909090909091,
  0.13953488372093023],
 'brevity_penalty': 1.0,
 'length_ratio': 1.1794871794871795,
 'translation_length': 46,
 'reference_length': 39}

Esses resultados representam a avaliação de um texto gerado usando a métrica BLEU. Cada componente do resultado oferece uma visão diferente da qualidade da tradução:

**bleu**: Esta é a pontuação BLEU geral do texto traduzido em relação ao(s) texto(s) de referência. A pontuação varia de 0 a 1, onde valores mais altos indicam melhor correspondência com o texto de referência. Uma pontuação de aproximadamente 0.19 sugere que, embora haja algum grau de correspondência, a qualidade do texto gerado tem bastante espaço para melhorias.

**precisions**: Estas são as precisões dos n-gramas de 1 a 4-gramas, respectivamente. A precisão para unigramas é 0.375, indicando que 37.5% dos unigramas na tradução coincidem com aqueles nos textos de referência. As precisões diminuem para n-gramas maiores, o que é esperado, pois é mais difícil conseguir correspondências exatas para sequências mais longas de palavras. A precisão mais baixa para 4-gramas (aproximadamente 0.133) sugere dificuldades em capturar estruturas de frases mais longas ou expressões idiomáticas corretamente.

**brevity_penalty**: A penalidade por brevidade (BP) ajusta a pontuação BLEU para traduções que são significativamente mais curtas que seus textos de referência. Uma BP próxima de 1, como neste caso, indica que a diferença de comprimento entre a tradução e o texto de referência é mínima, portanto, a penalidade aplicada é baixa.

**length_ratio**: Este é o ratio entre o comprimento da tradução e o comprimento do texto de referência. Um valor de aproximadamente 0.98 sugere que o comprimento da tradução é quase igual ao do texto de referência, corroborando a baixa brevidade da penalidade.

**translation_length**: O comprimento total da tradução gerada, medido em número de palavras.

**reference_length**: O comprimento total do texto de referência, também em número de palavras. A proximidade entre os comprimentos da tradução e da referência confirma que a penalidade por brevidade foi mínima.

Em resumo, os resultados sugerem que, embora o texto gerado tenha um comprimento adequado em comparação ao texto de referência e uma razoável correspondência de unigramas, há uma queda significativa na precisão à medida que se consideram sequências mais longas de palavras. Isso indica que, enquanto o texto gerado pode conter muitas palavras corretas individualmente, ela pode estar enfrentando dificuldades em capturar a estrutura correta das frases e a fluidez do texto de referência. Isso se deve ao baixo volume de dados usados no ajuste fino.

## Deploy e Uso do Modelo

In [46]:
# Define o device
device = next(modelo.parameters()).device

In [47]:
nova_pergunda_usuario = 'Como posso criar uma conta?'

In [48]:
pergunta_tokenizada = tokenizador(nova_pergunda_usuario, return_tensors = "pt", padding = True, truncation = True)

In [49]:
# Move o batch para o mesmo dispositivo do modelo
pergunta_tokenizada = {nome: tensor.to(device) for nome, tensor in pergunta_tokenizada.items()}

In [50]:
pergunta_tokenizada

{'input_ids': tensor([[33303, 45507, 20122,   270,  8569, 48185,    42]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [51]:
# Gera a previsão sem cálculo de gradiente e com autocast
with torch.no_grad(), torch.cuda.amp.autocast():
    previsao_tokens = modelo.generate(**pergunta_tokenizada, max_new_tokens = 50, pad_token_id = tokenizador.eos_token_id)

O autocast é um mecanismo do PyTorch para Mixed Precision Training que, ao ser ativado com o contexto faz com que o framework escolha automaticamente, para cada operador, se ele deve rodar em meia‑precisão (FP16) ou em precisão completa (FP32). Na prática, isso significa que as partes do cálculo que podem tolerar a menor precisão são executadas em FP16, o que reduz o uso de memória da GPU e acelera os kernels, enquanto operações mais sensíveis à perda de precisão continuam em FP32.

Para manter a estabilidade numérica durante o backward, o autocast costuma ser combinado com um GradScaler, que escala dinamicamente os gradientes para evitar underflow em FP16. Tudo isso é feito quase sem alterações no seu código de treinamento: basta envolver a passagem de forward no bloco autocast e, se desejar, emparelhar com o escalador de gradientes para garantir que a redução de precisão não afete a convergência do modelo.

In [52]:
# Decode da saída
tokenizador.decode(previsao_tokens[0], skip_special_tokens = True)

'Como posso criar uma conta?\nPara criar uma conta, você deve ter 18 anos ou mais.\nVocê pode criar uma conta no site ou no aplicativo.\nComo posso criar uma conta?\nPara criar uma conta, você deve ter '

In [53]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [54]:
#%watermark -v -m

In [55]:
#%watermark --iversions

# Fim